In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "6"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [3]:
import pandas as pd
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer

/home/worker/workspace/dialogue-summarization/.conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/worker/workspace/dialogue-summarization/.conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/worker/workspace/dialogue-summarization/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import random
import numpy as np
import torch
import re

def seed_everything(seed=10):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything()

In [5]:
m_name = "cointegrated/rut5-small"
tokenizer = T5Tokenizer.from_pretrained(m_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
import typing as tp

import sys

sys.path.append("/home/worker/workspace/hakatons/hakaton-gagarin-sentiment_interface/src")
from t5.utils import evaluate_metric, generate_answer_batched

EntityScoreType = tp.Tuple[int, float]  # (entity_id, entity_score)
MessageResultType = tp.List[EntityScoreType]  # list of entity scores,
#    for example, [(entity_id, entity_score) for entity_id, entity_score in entities_found]

In [19]:

model = torch.load('final_model.pth')



In [20]:
s = "США вводят санкции против российских банков:  Московский кредитный банк Банк «Урасиб» #USBN МТС Банк #MTSS Банк «Санкт-Петербург» #BSPB Банк «Зенит» Ланта Банк Металлинвестбанк Банк «Приморье» СДМ-Банк Уральский банк реконструкции и развития Банк ""Левобережный""  \xa0\xa0\xa0\xa0 — Frank Media  ⚠️🇺🇸🇷🇺#санкции #россия  Минфин США в новом пакете санкций против РФ вводит рестрикции против 22 физических лиц и 83 юрлиц.  Минфин США вводит санкции против предприятий металлургического сектора и горной промышленности РФ  Минфин США также вводит дополнительные санкции против предприятий ВПК РФ  \xa0\xa0\xa0\xa0\xa0 — ТАСС  💥🇷🇺#TCSG = +6%  Тинькофф также не вошел в список новых санкций\xa0 США  (Британия тоже не внесла)  #банки #санкции #россия"

df = pd.DataFrame({"message": [s]})

df["prefix"] = "clsorg"
df = df.rename({"message": "input_text"}, axis=1)



In [21]:
import evaluate
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from transformers import T5Tokenizer

# sys.path.append("/home/worker/workspace/hakatons/hakaton-gagarin-sentiment_interface/src")
from t5.model import NERModel

In [22]:
def generate_answer_batched(
    trained_model: NERModel,
    tokenizer: T5Tokenizer,
    data: pd.DataFrame,
    batch_size: int = 64,
):
    predictions = []
    with torch.no_grad():
        for name, batch in tqdm(data.groupby(np.arange(len(data)) // batch_size)):
            source_encoding = tokenizer(
                (batch["prefix"] + ": " + batch["input_text"]).tolist(),
                max_length=396,
                padding="max_length",
                truncation=True,
                return_attention_mask=True,
                add_special_tokens=True,
                return_tensors="pt",
            )

            generated_ids = trained_model.model.generate(
                input_ids=source_encoding["input_ids"].cuda(),
                attention_mask=source_encoding["attention_mask"].cuda(),
                num_beams=3,
                max_length=80,
                repetition_penalty=1.0,
                early_stopping=True,
                use_cache=True,
            ).cpu()

            preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            predictions.append(preds)

    return sum(predictions, [])

In [23]:
predictions = generate_answer_batched(
    trained_model=model, tokenizer=tokenizer, data=df, batch_size=64
)

100%|██████████| 1/1 [00:01<00:00,  1.09s/it]


In [25]:
entities_found = predictions

results = []

for row in entities_found:
    for entity in row.split(";"):
        t = []
        tup = entity.split('-')
        entity_id, entity_score = tup
        t.append((entity_id, entity_score))
    results.append(t)
    

In [26]:
results

[[('225', '2')]]

In [ ]:
def score_texts(
    messages: tp.Iterable[str], *args, **kwargs
) -> tp.Iterable[MessageResultType]:
    """
    Main function (see tests for more clarifications)
    Args:
        messages (tp.Iterable[str]): any iterable of strings (utf-8 encoded text messages)

    Returns:
        tp.Iterable[tp.Tuple[int, float]]: for any messages returns MessageResultType object
    -------
    Clarifications:
    >>> assert all([len(m) < 10 ** 11 for m in messages]) # all messages are shorter than 2048 characters
    """

    df = pd.DataFrame({"message": messages})

    entities_found = generate_answer_batched(
        trained_model=model, tokenizer=tokenizer, data=df, batch_size=64
    )

    for row in entities_found:
        for entity in row.split(";"):
            t = []
            tup = entity.split('-')
            entity_id, entity_score = tup
            t.append((entity_id, entity_score))
        results.append(t)

    return results